Import libraries necessary for this project

In [173]:
import numpy as np
import pandas as pd
from time import time
from IPython.display import display # Allows the use of display() for DataFrames

from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Pretty display for notebooks
%matplotlib inline

Below is a data ditionary from https://github.com/meauxt/credit-card-default/blob/master/credit_card_default.ipynb

Load the datasets

In [174]:
train = pd.read_csv("DataFiles/CreditCard_train.csv",header = 1)
test = pd.read_csv("DataFiles/CreditCard_test.csv",header = 1)
all_data = pd.concat([df_train,df_test])

In [175]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 5999
Data columns (total 25 columns):
ID                            30000 non-null int64
LIMIT_BAL                     30000 non-null int64
SEX                           30000 non-null int64
EDUCATION                     30000 non-null int64
MARRIAGE                      30000 non-null int64
AGE                           30000 non-null int64
PAY_0                         30000 non-null int64
PAY_2                         30000 non-null int64
PAY_3                         30000 non-null int64
PAY_4                         30000 non-null int64
PAY_5                         30000 non-null int64
PAY_6                         30000 non-null int64
BILL_AMT1                     30000 non-null int64
BILL_AMT2                     30000 non-null int64
BILL_AMT3                     30000 non-null int64
BILL_AMT4                     30000 non-null int64
BILL_AMT5                     30000 non-null int64
BILL_AMT6                

In [176]:
train.rename(columns={"PAY_0": "PAY_1"}, inplace = True)
test.rename(columns={"PAY_0": "PAY_1"}, inplace = True)

Split the train and test data for later training

In [151]:
train_y = train['default payment next month']
train_x = train.drop('default payment next month', axis = 1)
test_y = test['default payment next month']
test_x = test.drop('default payment next month', axis = 1)

In [152]:
general_stats= all_data.describe().iloc[:,0:6]
pay_status_stats = all_data.describe().iloc[:,6:12]
bill_stats = all_data.describe().iloc[:,12:18]
payed_stats = all_data.describe().iloc[:,18:24]

In [153]:
general_stats

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,15000.500000,167484.322667,1.603733,1.853133,1.551867,35.485500
std,8660.398374,129747.661567,0.489129,0.790349,0.521970,9.217904
min,1.000000,10000.000000,1.000000,0.000000,0.000000,21.000000
25%,7500.750000,50000.000000,1.000000,1.000000,1.000000,28.000000
50%,15000.500000,140000.000000,2.000000,2.000000,2.000000,34.000000
75%,22500.250000,240000.000000,2.000000,2.000000,2.000000,41.000000
max,30000.000000,1000000.000000,2.000000,6.000000,3.000000,79.000000


In [154]:
pay_status_stats

,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,-0.016700,-0.133767,-0.166200,-0.220667,-0.266200,-0.291100
std,1.123802,1.197186,1.196868,1.169139,1.133187,1.149988
min,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000
25%,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000


In [155]:
bill_stats

,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6
count,30000.000000,30000.000000,3.000000e+04,30000.000000,30000.000000,30000.000000
mean,51223.330900,49179.075167,4.701315e+04,43262.948967,40311.400967,38871.760400
std,73635.860576,71173.768783,6.934939e+04,64332.856134,60797.155770,59554.107537
min,-165580.000000,-69777.000000,-1.572640e+05,-170000.000000,-81334.000000,-339603.000000
25%,3558.750000,2984.750000,2.666250e+03,2326.750000,1763.000000,1256.000000
50%,22381.500000,21200.000000,2.008850e+04,19052.000000,18104.500000,17071.000000
75%,67091.000000,64006.250000,6.016475e+04,54506.000000,50190.500000,49198.250000
max,964511.000000,983931.000000,1.664089e+06,891586.000000,927171.000000,961664.000000


In [156]:
payed_stats

,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
count,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000
mean,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567
std,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775
min,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000
25%,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000
50%,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000
75%,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000
max,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000


Check if the data contains labels not included in the dictionary

In [157]:
for c in all_data.columns[2:12]:
    unique_vals = all_data[c].unique()
    print(c,"-"*5,unique_vals)

SEX ----- [2 1]
EDUCATION ----- [2 1 3 5 4 6 0]
MARRIAGE ----- [1 2 3 0]
AGE ----- [24 26 34 37 57 29 23 28 35 51 41 30 49 39 40 27 47 33 32 54 58 22 25 31
 46 42 43 45 56 44 53 38 63 36 52 48 55 60 50 75 61 73 59 21 67 66 62 70
 72 64 65 71 69 68 79 74]
PAY_0 ----- [ 2 -1  0 -2  1  3  4  8  7  5  6]
PAY_2 ----- [ 2  0 -1 -2  3  5  7  4  1  6  8]
PAY_3 ----- [-1  0  2 -2  3  4  6  7  1  5  8]
PAY_4 ----- [-1  0 -2  2  3  4  5  7  6  1  8]
PAY_5 ----- [-2  0 -1  2  3  5  4  7  8  6]
PAY_6 ----- [-2  2  0 -1  3  6  4  7  8  5]


In EDUCATION column, 5 and 6 both encodes for 'unknown'. There are 3 ways to eliminate this duplicate.
1.remove all 5s and 6s, and replace with Nan.
2.covert 5, 6 to 4, which encodes for 'others'.
3.keep the label 'unknown', convert 6 to 5.

In [134]:
# here I chose to convert 5,6 to 4 because the label 'others' does not contain more information than 'unknown'
train["EDUCATION"].replace([5,6], 4, inplace = True)
test["EDUCATION"].replace([5,6], 4, inplace = True)

In [143]:
#if we were to remove the unkown values, we should consider the amount of value we are removing
sum_unknown = all_data['EDUCATION'][(all_data['EDUCATION'] == 5)].count() + all_data['EDUCATION'][(all_data['EDUCATION'] == 6)].count()
sum_unknown/30000

0.011033333333333332

In [146]:
# all_data[['BILL_AMT1','PAY_AMT1']][(all_data['PAY_0'] == -2)]

,BILL_AMT1,PAY_AMT1
9,0,0
23,5512,19428
33,10929,4152
34,13709,5006
45,0,0
52,13465,7875
55,1905,3640
65,152519,0
92,-2000,7555
100,672,10212


In [98]:
clf = AdaBoostClassifier(random_state=41,n_estimators=15)
clf.fit(train_x,train_y)
accuracy = clf.score(test_x,test_y)
print (accuracy)

0.8285
